In [18]:
import fiddle as fdl
from fiddle.experimental import auto_config
from fiddle.codegen import codegen
from fiddle import printing, graphviz
from rich import inspect, print

from datasets.dataset import Dataset
from graphs.graph import Graph
from models.model import Model
from models.tasks.task import Task
from training.trainer import Trainer

In [10]:
@auto_config.auto_config
def make_trainer():
    """Creates a trainer.
    """
    dataset = Dataset(name="era", size=1000)
    # dataset should be of type Dataset, not str
    #dataset = 'ooops'
        graph = Graph(dataset=dataset, nodes=15)

    model = Model(graph=graph, hidden_dim=64)
    task = Task(model=model, task_type="classifier")
    trainer = Trainer(task=task, dataset=dataset, epochs=5)
    return trainer


cfg = make_trainer.as_buildable()

In [32]:
trainer = make_trainer()
trainer

Initialized MockDataset(name=era, size=1000)
Initialized MockGraph(nodes=15) with dataset=era
Initialized MockModel(hidden_dim=64) with graph linked to era
Initialized MockTask(task_type=classifier)
Initialized MockTrainer(epochs=5)


In [ ]:
print(cfg)

<Config[Trainer(
  task=<Config[Task(
    model=<Config[Model(
      graph=<Config[Graph(dataset=<Config[Dataset(name='era', size=1000)]>, nodes=15)]>,
      hidden_dim=64)]>,
    task_type='classifier')]>,
  dataset=<Config[Dataset(name='era', size=1000)]>,
  epochs=5)]>

In [ ]:
cfg.task.model.hidden_dim = 128
print(cfg)

<Config[Trainer(
  task=<Config[Task(
    model=<Config[Model(graph=128, hidden_dim=128)]>,
    task_type='classifier')]>,
  dataset=<Config[Dataset(name='era', size=1000)]>,
  epochs=5)]>

In [30]:
vis = graphviz.render(cfg)
print(vis)

graph {
        graph [overlap=false]
        node [fontname=Courier fontsize=10 margin=0 shape=none]
        edge [color="#00000030" penwidth=3]
        2 [label=<<table border="0" cellspacing="0" cellborder="1" cellpadding="3"><tr><td colspan="2" 
bgcolor="#fff8dc" style="solid"><font point-size="8">Config:</font>&nbsp;Model</td></tr><tr><td align="right" 
bgcolor="#eeeeee">graph</td><td align="left">128</td></tr><tr><td align="right" 
bgcolor="#eeeeee">hidden_dim</td><td align="left">128</td></tr></table>>]
        1:0:c -- 2:c [color="#cbc6b080"]
        1 [label=<<table border="0" cellspacing="0" cellborder="1" cellpadding="3"><tr><td colspan="2" 
bgcolor="#90ee90" style="solid"><font point-size="8">Config:</font>&nbsp;Task</td></tr><tr><td align="right" 
bgcolor="#eeeeee">model</td><td align="left"><table border="0" cellspacing="0" cellborder="1" cellpadding="3" 
style="solid"><tr><td port="0" bgcolor="#fff8dc" style="solid"></td></tr></table></td></tr><tr><td align="right" 
bgcolor="#eeeeee">task_type</td><td align="left">&#x27;classifier&#x27;</td></tr></table>>]
        0:1:c -- 1:c [color="#73be7380"]
        3 [label=<<table border="0" cellspacing="0" cellborder="1" cellpadding="3"><tr><td colspan="2" 
bgcolor="#ffa07a" style="solid"><font point-size="8">Config:</font>&nbsp;Dataset</td></tr><tr><td align="right" 
bgcolor="#eeeeee">name</td><td align="left">&#x27;era&#x27;</td></tr><tr><td align="right" 
bgcolor="#eeeeee">size</td><td align="left">1000</td></tr></table>>]
        0:2:c -- 3:c [color="#cb806180"]
        0 [label=<<table border="0" cellspacing="0" cellborder="1" cellpadding="3"><tr><td colspan="2" 
bgcolor="#ffc0cb" style="solid"><font point-size="8">Config:</font>&nbsp;Trainer</td></tr><tr><td align="right" 
bgcolor="#eeeeee">task</td><td align="left"><table border="0" cellspacing="0" cellborder="1" cellpadding="3" 
style="solid"><tr><td port="1" bgcolor="#90ee90" style="solid"></td></tr></table></td></tr><tr><td align="right" 
bgcolor="#eeeeee">dataset</td><td align="left"><table border="0" cellspacing="0" cellborder="1" cellpadding="3" 
style="solid"><tr><td port="2" bgcolor="#ffa07a" style="solid"></td></tr></table></td></tr><tr><td align="right" 
bgcolor="#eeeeee">epochs</td><td align="left">5</td></tr></table>>]
}

In [13]:
print(printing.as_str_flattened(cfg))

task.model.graph: Graph = 128
task.model.hidden_dim: int = 64
task.task_type: str = 'classifier'
dataset.name: str = 'era'
dataset.size: int = 1000
epochs: int = 5

In [14]:
generated = codegen.codegen_dot_syntax(cfg)
print("\n".join(generated.lines()))

from datasets import dataset
import fiddle as fdl
from models import model
from models.tasks import task
from training import trainer


def build_config():
  root = fdl.Config(trainer.Trainer)
  root.epochs = 5
  
  root.task = fdl.Config(task.Task)
  root.task.task_type = 'classifier'
  
  root.task.model = fdl.Config(model.Model)
  root.task.model.graph = 128
  root.task.model.hidden_dim = 64
  
  root.dataset = fdl.Config(dataset.Dataset)
  root.dataset.name = 'era'
  root.dataset.size = 1000
  
  return root